<a href="https://colab.research.google.com/github/bjdzliu/ai_lab/blob/main/langchain/milvus_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pymilvus

In [ ]:
!pip3 install grpcio==1.60.0


In [ ]:
from pymilvus import(
    connections,
    db,
    utility,
    FieldSchema,
    Collection,
    DataType,
    CollectionSchema

)

import os

os.environ['HF_TOKEN'] = "xxxxxx"

print(os.getenv('HF_TOKEN'))

In [14]:
connections.connect("default",host='xxxxxx',port='19530')
print(db.list_database())
print(utility.list_collections())


['default']
['demo_test']


In [ ]:
fields=[
    FieldSchema(name="pk",dtype=DataType.INT64,is_primary=True,auto_id=False),
    FieldSchema(name="words",dtype=DataType.VARCHAR,max_length=50),
    FieldSchema(name="embeddings",dtype=DataType.FLOAT_VECTOR,dim=512),
]

In [12]:
schema=CollectionSchema(fields,"simple demo for storing and receive from Google")

In [ ]:
from pymilvus import utility
utility.drop_collection("demo_test")

In [ ]:
demo=Collection('demo_test',schema)

In [ ]:
from transformers import AutoModel
from numpy.linalg import norm
model = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-small-en', trust_remote_code=True)

In [19]:
dog=model.encode(['dog'])

In [3]:
embeddings = model.encode(['dog','puppy','queen','book','lion'])

In [ ]:
print([dog])

In [4]:
data1=[
[1,2,3,4,5],
['dog','puppy','queen','book','lion'],
embeddings,
]


In [16]:
insert_result=demo.insert(data1)

In [17]:
index={

       "index_type":"FLAT",
       "metric_type":"L2",
       "params":{}
}
demo.create_index("embeddings",index)

Status(code=0, message=)

In [18]:
demo.load()

In [25]:
query_data=dog
search_parames={
        "metric_type":"L2",
    "params":{}
}
result=demo.search(query_data,"embeddings",search_parames,limit=5,output_fields=['words'])

In [26]:
result[0].ids

[1, 2, 5, 4, 3]

In [ ]:
result[0].distances

In [29]:
for i in range(0,5):
  hit=result[0][i]
  print(hit.entity.get('words'))

dog
puppy
lion
book
queen


按照pk编号来搜索

In [35]:
result2=demo.query(expr="pk>2",offset=0,limit=2,output_fields=["pk","words","embeddings"])

In [37]:
for i in range(2):
  hit = result2[i]
  print(hit.get('pk'))
  print(hit.get('words'))

3
queen
4
book
